# Данные

In [1]:
import csv
from collections import defaultdict
import random
import json
random.seed(42)

In [2]:
import sklearn.ensemble
import numpy as np

In [59]:
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

In [66]:
#Словари для основной выборки
user_to_items = defaultdict(set)
item_to_users = defaultdict(set)

#Словари для тестовой выборки
test_user_to_items = defaultdict(set)
test_item_to_users = defaultdict(set)



with open("data/train_likes.csv") as datafile:
    for like in csv.DictReader(datafile):
        # Кидаем монетку. В зависимости от результата кладём в обучение или тест
        if random.random() < 0.90:
            user_to_items[like['user_id']].add(like['item_id'])
            item_to_users[like['item_id']].add(like['user_id'])
        else:
            test_user_to_items[like['user_id']].add(like['item_id'])
            test_item_to_users[like['item_id']].add(like['user_id'])

In [67]:
all_items = set(item_to_users.keys()) | set(test_item_to_users.keys())
all_users = set(user_to_items.keys()) | set(test_user_to_items.keys())

In [73]:
matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
test_matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
for user, items in user_to_items.items():
    for item in items:
        matrix[user_to_i[user], item_to_i[item]] = True
for user, items in test_user_to_items.items():
    for item in items:
        test_matrix[user_to_i[user], item_to_i[item]] = True
matrix = matrix.tocsr()
test_matrix = test_matrix.tocsr()

In [74]:
matrix

<55863x23891 sparse matrix of type '<class 'numpy.float64'>'
	with 99676 stored elements in Compressed Sparse Row format>

In [75]:
u, s, vt = sp.sparse.linalg.svds(matrix.astype(np.float32), k=100)

# Фильтрация пользователей
* Значительная часть пользователей имеет всего 1-2 просмотра. При всём желании, рекоммендовать им что-либо осмысленное при помощи рассматриваемого здесь метода мы вряд ли сможем. Для простоты вычислений, удалим их из выборки.
* Важно понимать, что качество на оставшихся пользователях скорее всего будет выше, чем на первоначальной выборке.

In [56]:
min_items_per_user = 2
from copy import copy
for user in copy(test_user_to_items).keys():
    
    n_items_per_user = len(user_to_items[user]) + len(test_user_to_items[user])
    
    if n_items_per_user <= min_items_per_user:
        del user_to_items[user]
        del test_user_to_items[user]

In [58]:
user_to_i = {user: i for i, user in enumerate(all_users)}
item_to_i = {item: i for i, item in enumerate(all_items)}
all_users_list = list(all_users)
all_items_list = list(all_items)

### Рекоммендующая функция
Позволим себе немного вольности: наша функция будет возвращать не вероятности, а список фильмов в порядке убывания "рекомендованности".

* Рекоммендованность фильма item пользователю user посчитаем так:
  * Для каждого фильма, полайканного пользователем user, найдём других людей, которым понравился фильм.
  * Сложим всех таких "друзей по лайкам" вместе и назовём соседями (__neighborhood__) пользователя.
  * Для фильма item узнаем его аудиторию - множество пользователей, которые его лайкнули
  * Пригодность фильма пользователю - то, насколько "друзьям по лайкам" пользователя нравится этот фильм.

Для примера, будем использовать косинусную меру расстояния
  
$ cos(u_{film}, u_{neighborhood}) = $ =$ u_{film} \cdot u_{neighborhood} \over |u_{film}| |u_{neighborhood}| $


$u_{neighborhood}$ зависит только от пользователя, но не от фильма, поэтому при сравнении фильмов по пригодности для одного пользователя, его можно исключить из формулы для простоты вычислений.

$ similarity(u_{film}, u_{neighborhood}) = $ $  u_{film} \cdot u_{neighborhood} \over |u_{film}| $
  
  
Распишем формулу подробно:

$ similarity(u_{film}, u_{neighborhood}) = $ $ \sum _{u_i} [u_i \in u_{film}] \cdot [u_i \in u_{neighborhood}] \over |u_{film}|  $

* u_i - очередной пользователь (в цикле по всем пользователям)
  
Выражение $[u_i \in u_{neighborhood}]$ здесь означает "сколько раз очередной пользователь входит в множество друзей по лайкам"
  
  

In [90]:
from math import sqrt
from collections import Counter

def recommend(user, n_best = 10):
    user_items = user_to_items[user]
    
    neighborhood = Counter()
    for item in user_items:
        neighborhood.update(item_to_users[item])
    
    #словарь {фильм -> пригодность фильма пользователю}
    item_similarities = {}
    
    for item in all_items:
        if item in user_items: continue
        item_users = item_to_users[item]
        if len(item_users) == 0: continue
        
        n_common_users = sum(neighborhood[user] for user in item_users)
        similarity = float(n_common_users) / sqrt(len(item_users))
        item_similarities[item] = similarity
    
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    
    return items_sorted[:n_best]

In [91]:
user_to_int = dict()
for i, u in enumerate(all_users):
    user_to_int[u] = i 

In [9]:
films = json.load(open('data/items.json'))
films = {a['id']:a for a in films}

In [34]:
#Making dataset
user_features = defaultdict(lambda:defaultdict(lambda:0))
for u in list(user_to_items.keys())[0:]:
    for item in user_to_items[u]:
        if item in films:
            for feature, value in films[item].items():
                if feature != 'id':
                    if feature=='genre':
                        user_features[u][value] += 1
                    else:    
                        user_features[u][feature]+=value
    for f in user_features[u]:
        user_features[u][f]/=len(user_to_items)

In [78]:
def user_film_mk1(user, item):
    #print(len(user_features[user]))
    cursum = 0
    curcnt = 0
    if item in films:
        for feature, value in films[item].items():
            if type(value) is int:
                curcnt+=1
                if feature=='genre':
                    #print(user_features[user])
                    cursum+=user_features[user][value]
                else:    
                    cursum+=value*user_features[user][feature]
    if curcnt==0:
        curcnt+=1
    #cursum/=curcnt
    #print(cursum)
    return cursum

In [79]:
def user_film_mk2(user, item):
    ui = user_to_i[user]
    ii = item_to_i[item]
    a = np.dot(u[ui,:] * s, vt[:,ii])
    return a

In [19]:
# Порекоммендуем топ-5 фильмов какому-то юзверю
recommend('d8c2794b01531ca807bc2b28d171f22d', n_best=5)

['e6e53f41066b37fb5b80bd118dc800be',
 '44327280355abfc1c58fa9ad8c41a2cc',
 '8cb44b2217dd1cc509465a07cbb68152',
 'd5ba3ed09490eb5bc1b92a87ea99e227',
 '79c029bfc86d2566e44e816a9dfc4192']

In [81]:
user_film_mk2('17db499bbbd058da91bddcaea365be5c', '98bdfa4bcad95291e88d56521b620acc')

0.034973167

In [84]:
def generic_recommend(user_flim_function):
    def recommend(user, n_best = 10, debug=False):
        user_items = user_to_items[user]

        item_similarities = {}
        for item in all_items:
            if item in user_items: continue
            item_users = item_to_users[item]
            if len(item_users) == 0: continue

            item_similarities[item] = user_flim_function(user, item)

        items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
        if debug:
            for a in  items_sorted[:n_best]:
                print((a, item_similarities.get(a, 0)))
        return items_sorted[:n_best]
    return recommend

In [86]:
generic_recommend(user_film_mk2)('bc7040d6170f80a4dd7c116161648588', debug=True)

('71af070804f8f2fd21a56e7b6280760c', 0.096845694)
('ea0f84f4dec3c66c5cdc99e793bde265', 0.05196017)
('77146c61cf8784ec3f71575fec5a1a91', 0.046646461)
('7e4f9ce46b56cc4069fe29df91cf2678', 0.04476697)
('3131eaaa5222e2ab5a15527b4d541386', 0.043919683)
('5868853865709a004ec25f2f61b86bde', 0.039338164)
('69746dd9b05454baef427877a7676da1', 0.035707682)
('f454a1aa6d742b61308b22baf4301304', 0.035558354)
('ded9b9e2dd5d3e23c97b2993dfa46bf5', 0.034418773)
('1effb78d80bd8fc7d4cab1161a34b255', 0.030028159)


['71af070804f8f2fd21a56e7b6280760c',
 'ea0f84f4dec3c66c5cdc99e793bde265',
 '77146c61cf8784ec3f71575fec5a1a91',
 '7e4f9ce46b56cc4069fe29df91cf2678',
 '3131eaaa5222e2ab5a15527b4d541386',
 '5868853865709a004ec25f2f61b86bde',
 '69746dd9b05454baef427877a7676da1',
 'f454a1aa6d742b61308b22baf4301304',
 'ded9b9e2dd5d3e23c97b2993dfa46bf5',
 '1effb78d80bd8fc7d4cab1161a34b255']

In [46]:
recommend('17db499bbbd058da91bddcaea365be5c')

['a79593d2e2054ce444f75e709a2610ee',
 '6e2d20f2525c74d524ab385481580428',
 'b2b167ac5d130cd19870fa83c74e2e14',
 'ac4c19bcd373e8d670b87738599f1a33',
 '8c472f83e541fa461dd92a4c4d6b164e',
 'a6188673adec0aa69b5e6e6b38019882',
 '6d2409eb21e08e782651f99d1995e0bd',
 'fb9e753ca2560b521c8ffb443c355450',
 'b3a59f101d03efb903b6fc914a6a9472',
 '0092a1ce5b8f0195dd2d8676b1159040']

In [23]:
list(test_user_to_items.items())[1]

('bc7040d6170f80a4dd7c116161648588', {'99790dabab12acee864a30848fbc8ad3'})

In [24]:
def recommend_dummy(user, n_best = 10):
    item_similarities = []
    for item in all_items:
        #пропустим те фильмы, которые пользователь уже просмотрел, если нас об этом попросили
        if item in user_to_items[user]: continue
        item_similarities.append(item)
         
    random.shuffle(item_similarities)
    #вернём n_best наиболее пригодных
    #print(items_sorted[:n_best])
    return item_similarities[:n_best]

# Оценка качества - map@k

In [28]:
check_quality(recommend_dummy,10,200)

0 / 500
100 / 500
200 / 500
300 / 500
400 / 500
AP@10 = 0.000163015873015873


In [87]:
check_quality(generic_recommend(user_film_mk1),10,200)

0 / 200
100 / 200
AP@10 = 0.13951865079365053


In [88]:
check_quality(generic_recommend(user_film_mk2),10,200)

0 / 200
100 / 200
AP@10 = 0.004848611111111112


In [89]:
check_quality(recommend,10,200)

0 / 200
100 / 200
AP@10 = 0.0013873015873015872


In [92]:
len(test_user_to_items)

8324

In [26]:
def APatK (actual_likes,recommendation_list, K =10):
    """Посчитать Average Precision at K"""

    countRelevants = 0
    sum_of_precisions = 0.0
    
    for i in range(min(K,len(recommendation_list))):
        currentk = i + 1.0
        if recommendation_list[i] in actual_likes:
            countRelevants+=1
        precisionAtK = countRelevants / currentk 
        sum_of_precisions += precisionAtK
        
    return sum_of_precisions / K 

In [27]:
#сколько рекоммендаций рассматриваем
def check_quality(function, K = 10, max_n_users = 500):
    APatK_per_user = []
    user_list = list(test_user_to_items.keys())[:max_n_users]
    for i, user in enumerate(user_list):
        #фильмы, которые пользователю на самом деле нравятся
        test_items = test_user_to_items[user]

        #Выдать топ-K рекоммендаций
        recommendation_list = function(user,n_best=K)

        #Посчитать ap@k
        user_APatK = APatK(test_items, recommendation_list,K=K)

        #и сложить в коробку
        APatK_per_user.append(user_APatK)

        #Progress bar
        if i % 100 ==0:
            print(i,'/',max_n_users)

        if i > max_n_users:
            break

    print('AP@{} = {}'.format(K, sum(APatK_per_user)/len(APatK_per_user)))


# Notes
* Кроме качества рекоммендаций, map@k ещё зависит от доли тестовой выборки, фильтрации и от самого K. Сравнивать качество разных алгоритмов имеет смысл только при одинаковом K и тестовой выборке.
* Давать полезные рекоммендации пользователям с малым числом просмотров тоже можно: например, можно выдавать наиболее популярные в целом фильмы.
* Разделение на обучение/тест честнее делать на по времени: первые 70% (например) лайков в обучение, остальные в тест. Это ближе к реальной жизни, когда вы сначала обучаете модель на логах, а потом применяете на новых сессиях пользователей.